In [2]:
# Script takes all .cif strucutres and converts them to 3x3x3 supercells

import glob
from pymatgen.io.cif import CifWriter
from pymatgen.core.structure import Structure

unit = glob.glob('*.cif')
unit.sort()

###### Set parameter ######
size = 7 #NxNxN
###########################

for u in unit:
    structure = Structure.from_file(u)
    structure.make_supercell([size,size,size])
    w = CifWriter(structure)
    w.write_file(f'{u[:-4]}_{size}x{size}x{size}.cif')

/home/ben/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1126: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


In [4]:
# Script to convert supercells to .lmp

import glob
import subprocess

supers = glob.glob(f'*{size}.cif')
supers.sort()

for s in supers:
    process = subprocess.run(['atomsk', s, '-properties', 'charges.txt', 'lammps'])

In [5]:
# Script to create .inp file

import glob
import random

###### Set parameter ######
temps = [300] #K
timestep = 0.001 #fs
length = 10000000 #fs
###########################

structures = glob.glob('*.lmp')
structures.sort()

for structure in structures:
    for temp in temps:
        filename = (structure[:-4] + "_" + str(temp) + "k.inp")
        target = open(filename, 'w+')
        target.write(f"""
# Input written to comply with LAMMPS 2017-08-11 00:00:00

# Atomic Configuration
units           metal
boundary        p p p
box             tilt large
atom_style      charge
read_data       {structure}
group           lithium type 1
group           scandium type 2
group           chloride type 3
variable        T1 equal {temp}
variable        dt equal {timestep}

# Potential Setup
pair_style      buck/coul/long 12.0
pair_coeff      * * 0.0 1.0 0.0
pair_coeff      1 1    360.530000  0.161000   0.000000 # Li Li
pair_coeff      1 3    421.040000  0.344000   0.000000 # Li Cl
pair_coeff      3 3   1227.200000  0.321000  14.530000 # Cl Cl
pair_coeff      2 3  12131.350000  0.250700   0.000000 # Sc Cl

kspace_style    pppm 1e-05

# General Setup
timestep        ${{dt}}

# Thermodynamic Information Output
thermo_style    custom step temp epair emol etotal press lx ly lz fmax cpu cpuremain
thermo          1000

# Intial Atom Velocity
velocity        all create ${{T1}} {random.randrange(1000000)} dist gaussian rot yes

# Stage 0: npt_equilibriation
restart         0
fix             int all npt aniso 1.0 1.0 $(1000*dt) temp ${{T1}} ${{T1}} $(100*dt) 
run             50000

# Stage 1: nvt_equilibriation
unfix           int
restart         0
fix             int all nvt temp ${{T1}} ${{T1}} $(100*dt) 
run             6000000

# Stage 2: main_nvt_simulation
unfix           int
compute         lithiummsd lithium msd com yes
compute         scandiummsd scandium msd com yes
compute         chloridemsd chloride msd com yes
fix             lithiummsdt lithium ave/time 1 1 1000 c_lithiummsd[4] file {structure[:-4]}_{str(temp)}_li.txt
fix             scandiummymsdt oxide ave/time 1 1 1000 c_scandiummsd[4] file {structure[:-4]}_{str(temp)}_sc.txt
fix             chloridemymsdt chloride ave/time 1 1 1000 c_chloridemsd[4] file {structure[:-4]}_{str(temp)}_cl.txt
dump            atom_info all custom 1000 {structure[:-4]}_{str(temp)}.lammpstrj element x y z q id type vx vy vz
dump_modify     atom_info append yes
dump_modify     atom_info sort id
dump_modify     atom_info element Li Sc Cl
restart         0
fix             int all nvt temp ${{T1}} ${{T1}} $(100*dt) 
run             {length}

# Final Commands
variable        final_energy equal etotal
print           "final_energy: ${{'final_energy'}}"
print           "END_OF_COMP"
""")

In [4]:
# Script to create .sh file

import glob

inputs = glob.glob('*.inp')
inputs.sort()

###### Set parameter ######
cores = 160
time = str('48:00:00')
###########################

for inp in inputs:
    filename = (inp[:-3] + "sh")
    target = open(filename, 'w+')
    target.write(f"""
#!/bin/bash
#$ -cwd
#$ -S /bin/bash
#$ -m n
#$ -N {inp[0:3]}
#$ -V
#$ -o _scheduler-stdout.txt
#$ -e _scheduler-stderr.txt
#$ -P Gold
#$ -A Faraday_CATMAT
#$ -pe mpi {cores}
#$ -l mem=314572K
#$ -l h_rt={time}
#$ -ac allow=A

module load lammps/7Aug19/userintel/intel-2018

'gerun' '/shared/ucl/apps/lammps/7Aug2019/USERINTEL/intel-2018/lammps/bin/lmp_default' '-in' '{inp}'
""")

In [10]:
# Script creates folders on Michael and moves files into folders

for u in unit:
    for temp in temps:
        process = subprocess.run(['ssh', 'michael', 'mkdir', f'halospinel/long/{u[:-4]}'])
        process2 = subprocess.run(['scp', f'{u[:-4]}_{size}x{size}x{size}.lmp', f'michael:halospinel/long/{u[:-4]}'])
        process3 = subprocess.run(['scp', f'{u[:-4]}_{size}x{size}x{size}_{temp}k.inp', f'michael:halospinel/long/{u[:-4]}'])
        process4 = subprocess.run(['scp', f'{u[:-4]}_{size}x{size}x{size}_{temp}k.sh', f'michael:halospinel/long/{u[:-4]}'])

In [ ]:
# Script runs calculations on Michael

for u in unit:
    for temp in temps:
        process = subprocess.run(['ssh', 'michael', 'qsub', f'halospinel/calcs/{u[:-4]}/{u[:-4]}_{size}x{size}x{size}_{temp}k.sh'])

In [8]:
print(unit)

['monoclinic.cif', 'triclinic.cif', 'z_dummy.cif']
